In [3]:
!conda env list

# conda environments:
#
                         C:\Users\raysu\Anaconda3
RL_env                   C:\Users\raysu\Anaconda3\envs\RL_env
base_py_36               C:\Users\raysu\Anaconda3\envs\base_py_36
kedro_mlflow_env         C:\Users\raysu\Anaconda3\envs\kedro_mlflow_env
ml_env_py37              C:\Users\raysu\Anaconda3\envs\ml_env_py37
point_cloud_env          C:\Users\raysu\Anaconda3\envs\point_cloud_env
sphynx_env               C:\Users\raysu\Anaconda3\envs\sphynx_env
test_env                 C:\Users\raysu\Anaconda3\envs\test_env
tf2_env_py36             C:\Users\raysu\Anaconda3\envs\tf2_env_py36
                         C:\Users\raysu\Documents\GitHub\test_flask\.env
base                     c:\Users\raysu\Anaconda3
RL_env                   c:\Users\raysu\Anaconda3\envs\RL_env
base_py_36               c:\Users\raysu\Anaconda3\envs\base_py_36
fav_env                  c:\Users\raysu\Anaconda3\envs\fav_env
kedro_mlflow_env         c:\Users\raysu\Anaconda3\envs\kedro_mlflow_env
ml_e

In [5]:
import numpy as np
import pandas as pd

In [38]:
from io import StringIO
from jinja2 import Template, Environment, PackageLoader
import yaml

In [39]:
s = """
# Instruction:
## For an environment installed locally
# cd ./.env
# mkdir venv && cd venv
# conda env create --prefix . -f ../environment.yml
## For Updating local environment
# conda env update --prefix . -f ../environment.yml  --prune
#
## For an environment installed globally
## with a name: fav_env
# conda env create -f environment.yml

name: fav_env # favorite env
channels:
  - conda-forge
  - anaconda
  - default
dependencies:
  - python=3.7
  - numpy
  - pandas
  - scipy
  - matplotlib
  - plotly
  - jupyter
  - jupyterlab
  - tqdm
  - flask # -c conda-forge, anaconda
  - flask-wtf # -c conda-forge, anaconda
  - werkzeug==0.16.1   # https://github.com/pallets/flask/issues/3481#issuecomment-583293364
  - requests
  - codecov
  - pip
  - sphinx
  - pandoc # -c conda-forge
  - tensorflow
  - scikit-learn
  - scikit-image
  - imageio
  - pylint
  - autopep8
  - beautifulsoup4 # -c conda-forge
  - kaggle # -c conda-forge
  - pip:
    - Flask-Testing
    - hypothesis
    - sphinx_rtd_theme
"""

In [44]:
yaml_data = yaml.load(StringIO(s), Loader=yaml.FullLoader)
yaml_data

{'name': 'fav_env',
 'channels': ['conda-forge', 'anaconda', 'default'],
 'dependencies': ['python=3.7',
  'numpy',
  'pandas',
  'scipy',
  'matplotlib',
  'plotly',
  'jupyter',
  'jupyterlab',
  'tqdm',
  'flask',
  'flask-wtf',
  'werkzeug==0.16.1',
  'requests',
  'codecov',
  'pip',
  'sphinx',
  'pandoc',
  'tensorflow',
  'scikit-learn',
  'scikit-image',
  'imageio',
  'pylint',
  'autopep8',
  'beautifulsoup4',
  'kaggle',
  {'pip': ['Flask-Testing', 'hypothesis', 'sphinx_rtd_theme']}]}

In [74]:
str(list)

"<class 'list'>"

In [139]:
list(yaml_data.keys())[0]

'name'

In [164]:
yaml_data.keys(), list(yaml_data['dependencies'][-1].values())[0]

(dict_keys(['name', 'channels', 'dependencies']),
 ['Flask-Testing', 'hypothesis', 'sphinx_rtd_theme'])

In [174]:
subfield = Field(name = yaml_data['dependencies'][-1], value=None, parent='dependencies')
print(subfield)

{
  "field_name": "dependencies_pip",
  "field_value": [
    "Flask-Testing",
    "hypothesis",
    "sphinx_rtd_theme"
  ],
  "parent": "dependencies",
  "dtype": "list"
}


In [173]:
import copy, json
class Field(object):
    def __init__(self, name=None, value=None, parent=None):
        self.name = name
        self.value = value
        self.parent = parent
        self._update_dtype()
        self._update_name()
        
    def __repr__(self):
        return "a 'Field' class object"
    
    def __str__(self, indent=2):
        return json.dumps({'field_name': self.name, 
                           'field_value': self.value, 
                           'parent': self.parent, 
                           'dtype': self.dtype}, 
                          indent=indent)
        
    def _update_dtype(self):    
        self._dtypes = {'list': list, 'dict': dict, 'str': str}
        if (self.value is None):
            check = self.name
        else:
            check = self.value
        #print(check)
        #check = copy.deepcopy(self.name) if (self.value is None) else copy.deepcopy(self.value)
        self.dtype = [key for key, val in self._dtypes.items() if isinstance(check, val)][0]        
        
    def _update_name(self):
        if self.dtype=='dict':
            prefix = f'{self.parent}_' if (self.parent is not None) else ''
            name, value = prefix + list(self.name.keys())[0], list(self.name.values())[0]
            self.name, self.value = name, value
            #print((name,value))
            self._update_dtype()
            #self.dtype = [key for key, val in self._dtypes.items() if isinstance(value, val)][0]
            

In [80]:
fields = list()
for field_name, field_value in yaml_data.items():
    fields.append(Field(name=field_name, value=field_value, parent=None))

In [177]:
template = '''\n
<html>
<body>
{% for field_name, field_value in fields.items() %}
    {% set field = Field(name=field_name, value=field_value, parent=None) %}
    {% if field.dtype=="str" %}
        {{ field.name }}: {{ field.value }} <br/>
    {% else %}
        {% set nitems = LEN(field.value) %}
        {% set ndigits = LEN(STR(nitems)) %}
        {{ field.name }}: Total {{ nitems }} subfields. <br/>
        <!--- <tab/> {{ "SL#" }}  <tab/>  {{ "SUBFIELD VALUE" }} <br/> --->
        {% for idx, item in ENUMERATE(field.value) %}
            {% set subfield = Field(name=item, value=None, parent=field.name) %}
            {% if subfield.dtype!="list" %}
                {{ " |---" }} {{ STR(idx).zfill(ndigits) }}. {{ subfield.name }} <br/>
            {% elif subfield.dtype=="list" %}
                {% set nndigits = LEN(STR(LEN(subfield.value))) %}
                {% for iidx, iitem in ENUMERATE(subfield.value) %}
                    {% set subfield = Field(name=iitem, value=None, parent=field.name) %}              
                    {{ " |---" }} {{ STR(iidx).zfill(ndigits) }}. {{ subfield.name }} <br/>
                {% endfor %}
            {% endif %}            
        {% endfor %}
    {% endif %}
{% endfor %}
</body>
</html>
'''

In [68]:
isinstance([1,2], list)

True

In [100]:
from IPython.display import display, IFrame, HTML

In [178]:
tm = Template(template, autoescape=False)
msg = tm.render(fields=yaml_data, Field = Field, LEN = len, STR = str, ENUMERATE = enumerate)

#print(msg)

In [179]:
HTML(msg)

In [27]:
x = np.arange(10)
y = 3*x**2 + 5*x - 6

In [12]:
df = pd.DataFrame({'x': x, 'y': y})

In [13]:
df

,x,y
0,0,-6
1,1,2
2,2,16
3,3,36
4,4,62
5,5,94
6,6,132
7,7,176
8,8,226
9,9,282


In [17]:
df.agg({
    'x': ['mean', 'std'], 
    'y': ['mean', 'median']
       }
)

,x,y
mean,4.50000,102.0
median,NaN,78.0
std,3.02765,NaN


In [19]:
print(df.agg.__doc__)


Aggregate using one or more operations over the specified axis.

.. versionadded:: 0.20.0

Parameters
----------
func : function, str, list or dict
    Function to use for aggregating the data. If a function, must either
    work when passed a DataFrame or when passed to DataFrame.apply.

    Accepted combinations are:

    - function
    - string function name
    - list of functions and/or function names, e.g. ``[np.sum, 'mean']``
    - dict of axis labels -> functions, function names or list of such.
axis : {0 or 'index', 1 or 'columns'}, default 0
        If 0 or 'index': apply function to each column.
        If 1 or 'columns': apply function to each row.
*args
    Positional arguments to pass to `func`.
**kwargs
    Keyword arguments to pass to `func`.

Returns
-------
scalar, Series or DataFrame

    The return can be:

    * scalar : when Series.agg is called with single function
    * Series : when DataFrame.agg is called with a single function
    * DataFrame : when DataFram

In [20]:
pd.__version__

'1.0.1'

In [21]:
df.transform?

Signature: df.transform(func, axis=0, *args, **kwargs) -> 'DataFrame'
Docstring:
Call ``func`` on self producing a DataFrame with transformed values.

Produced DataFrame will have same axis length as self.

Parameters
----------
func : function, str, list or dict
    Function to use for transforming the data. If a function, must either
    work when passed a DataFrame or when passed to DataFrame.apply.

    Accepted combinations are:

    - function
    - string function name
    - list of functions and/or function names, e.g. ``[np.exp. 'sqrt']``
    - dict of axis labels -> functions, function names or list of such.
axis : {0 or 'index', 1 or 'columns'}, default 0
    If 0 or 'index': apply function to each column.
    If 1 or 'columns': apply function to each row.
*args
    Positional arguments to pass to `func`.
**kwargs
    Keyword arguments to pass to `func`.

Returns
-------
DataFrame
    A DataFrame that must have the same length as self.

Raises
------
ValueError : If the retu

In [37]:
print(*[1,2,3])
params = {'xvals': x, 'yvals': y}
[e for e in params.items()]

1 2 3


[('xvals', array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])),
 ('yvals', array([ -6,   2,  16,  36,  62,  94, 132, 176, 226, 282]))]

In [ ]:
df.transform()